In [1]:
!pip install --upgrade transformers


In [2]:
!ls /content/drive/MyDrive/Model_num1


ls: cannot access '/content/drive/MyDrive/Model_num1': No such file or directory


In [3]:
from transformers import AutoTokenizer, CodeGenForCausalLM, CodeGenConfig

model_path = "/content/drive/MyDrive/Model_num1"  # Your existing model folder

tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
config = CodeGenConfig.from_pretrained("Salesforce/codegen-350M-mono")
config.vocab_size = len(tokenizer)

model = CodeGenForCausalLM.from_pretrained(
    model_path,
    config=config,
    local_files_only=True,
    ignore_mismatched_sizes=True,
)

print("✅ Fine-tuned model and tokenizer loaded successfully.")


HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/content/drive/MyDrive/Model_num1'. Use `repo_type` argument if needed.

In [4]:
!ls -l /content/drive/MyDrive/Model_num1


ls: cannot access '/content/drive/MyDrive/Model_num1': No such file or directory


In [5]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
from transformers import AutoTokenizer, CodeGenForCausalLM, CodeGenConfig

model_path = "/content/drive/MyDrive/Model_num1"  # Your existing model folder

tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
config = CodeGenConfig.from_pretrained("Salesforce/codegen-350M-mono")
config.vocab_size = len(tokenizer)

model = CodeGenForCausalLM.from_pretrained(
    model_path,
    config=config,
    local_files_only=True,
    ignore_mismatched_sizes=True,
)

print("✅ Fine-tuned model and tokenizer loaded successfully.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

Some weights of CodeGenForCausalLM were not initialized from the model checkpoint at /content/drive/MyDrive/Model_num1 and are newly initialized because the shapes did not match:
- lm_head.bias: found shape torch.Size([51200]) in the checkpoint and torch.Size([50295]) in the model instantiated
- lm_head.weight: found shape torch.Size([51200, 1024]) in the checkpoint and torch.Size([50295, 1024]) in the model instantiated
- transformer.wte.weight: found shape torch.Size([51200, 1024]) in the checkpoint and torch.Size([50295, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Fine-tuned model and tokenizer loaded successfully.


In [7]:
prompt = """def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)
"""

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

output_ids = model.generate(
    input_ids,
    max_new_tokens=50,
    do_sample=True,
    temperature=0.2,
    top_p=0.95,
    num_beams=5,
    early_stopping=True,
    pad_token_id=tokenizer.eos_token_id,
)

generated_code = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("=== Generated raw code ===")
print(generated_code)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


=== Generated raw code ===
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)
�elling questionnaireStreamerBotoreaserving Morris Oaks STEP rollsocation referees firms Grande abnormalpoons Bradford Ain tabloreaserving Wyatt Wyatt lunch Lizard rackedUV porkafi facilitated Alexis Davies472 represent arts currencyUnitedizons applicants STEPDefense abrasaylor businesses convoluted asymm Osh Jensen businessesuki


In [8]:
def extract_clean_function(code_str, func_name="factorial"):
    lines = code_str.splitlines()
    func_lines = []
    inside_func = False

    for line in lines:
        if line.strip().startswith(f"def {func_name}"):
            inside_func = True
        if inside_func:
            if line.startswith("def ") or line.startswith("    "):
                func_lines.append(line)
            else:
                break

    return "\n".join(func_lines)

clean_code = extract_clean_function(generated_code)

print("\n=== Extracted clean code ===")
print(clean_code)



=== Extracted clean code ===
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)


In [9]:
test_cases = [(0, 1), (1, 1), (3, 6), (5, 120)]

def evaluate_model_code(code_str, func_name, test_cases):
    try:
        local_env = {}
        exec(code_str, local_env)
        func = local_env[func_name]
    except Exception as e:
        print("⚠️ Error executing code:", e)
        return 0.0

    passed = 0
    for inp, expected in test_cases:
        try:
            if func(inp) == expected:
                passed += 1
        except Exception as e:
            print(f"    ❌ Error on input {inp}:", e)
    return passed / len(test_cases)

reward = evaluate_model_code(clean_code, "factorial", test_cases)
print(f"\n🎯 Model success rate on tests: {reward * 100:.1f}%")



🎯 Model success rate on tests: 100.0%


In [10]:
tests = [
    {
        "prompt": """def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)
""",
        "func_name": "factorial",
        "test_cases": [(0, 1), (1, 1), (3, 6), (5, 120)],
    },
    {
        "prompt": """def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)
""",
        "func_name": "fibonacci",
        "test_cases": [(0, 0), (1, 1), (5, 5), (7, 13)],
    },
    {
        "prompt": """def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True
""",
        "func_name": "is_prime",
        "test_cases": [(2, True), (4, False), (13, True), (15, False)],
    },
    {
        "prompt": """def reverse_string(s):
    return s[::-1]
""",
        "func_name": "reverse_string",
        "test_cases": [("abc", "cba"), ("", ""), ("Hello", "olleH")],
    },
]


In [11]:
def test_function(model, tokenizer, prompt, func_name, test_cases):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
    output_ids = model.generate(
        input_ids,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.2,
        top_p=0.95,
        num_beams=5,
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id,
    )
    generated_code = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Extract clean function
    def extract_clean_function(code_str, func_name=func_name):
        lines = code_str.splitlines()
        func_lines = []
        inside_func = False
        for line in lines:
            if line.strip().startswith(f"def {func_name}"):
                inside_func = True
            if inside_func:
                if line.startswith("def ") or line.startswith("    "):
                    func_lines.append(line)
                else:
                    break
        return "\n".join(func_lines)

    clean_code = extract_clean_function(generated_code)

    print(f"\n=== Generated code for function {func_name} ===")
    print(clean_code)

    # Evaluate function
    def evaluate_model_code(code_str, func_name, test_cases):
        try:
            local_env = {}
            exec(code_str, local_env)
            func = local_env[func_name]
        except Exception as e:
            print(f"⚠️ Error executing function {func_name}:", e)
            return 0.0
        passed = 0
        for inp, expected in test_cases:
            try:
                if func(inp) == expected:
                    passed += 1
            except Exception as e:
                print(f"    ❌ Error on input {inp}:", e)
        return passed / len(test_cases)

    reward = evaluate_model_code(clean_code, func_name, test_cases)
    print(f"🎯 Success rate for {func_name}: {reward * 100:.1f}%")
    return reward


In [ ]:
total_tests = 100
repeats_per_test = total_tests // len(tests)
results = []

for test in tests:
    for _ in range(repeats_per_test):
        reward = test_function(model, tokenizer, test["prompt"], test["func_name"], test["test_cases"])
        results.append({
            "func_name": test["func_name"],
            "reward": reward
        })



=== Generated code for function factorial ===
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)
🎯 Success rate for factorial: 100.0%

=== Generated code for function factorial ===
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)
🎯 Success rate for factorial: 100.0%


In [ ]:
import pandas as pd

df = pd.DataFrame(results)
summary = df.groupby("func_name")["reward"].mean().reset_index()
summary["reward_percent"] = summary["reward"] * 100

print("\n=== Summary of Test Results ===")
print(summary)
